# Check implementation of Hamberg et al model

In [1]:
import os

import chi
import chi.plots
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go

from model import define_hamberg_model, define_hamberg_population_model

## Define Hamberg model

In [2]:
mechanistic_model, parameters_df = define_hamberg_model()

## Define covariate model (ignoring other sources of IIV)

In [3]:
def get_typical_parameters(age, cyp2c9, vkorc1):
    """
    Returns typical model parameters for patient with input characteristics
    according to Hamberg et al.

    :param age: The age of the patient.
    :type age: int
    :param cyp2c9: One of 5 variants of the CYP2C9 gene:
        ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    :type cyp2c9: str
    :param vkorc1: One of 5 variants of the VKORC1 gene:
        ['GG', 'GA', 'AA']
    :type vkorc1: str
    """
    cyp2c9_variants = ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    if cyp2c9 not in cyp2c9_variants:
        raise ValueError(
            'The provided CYP2C9 genotype is not among the allowed variants.')
    vkorc1_variants = ['GG', 'GA', 'AA']
    if vkorc1 not in vkorc1_variants:
        raise ValueError(
            'The provided VKORC1 genotype is not among the allowed variants.')

    # Define covariate independent parameters
    effective_volume_central = 14.3           # in L
    transition_rate_chain_1 = 3 / 28.6        # in 1/h
    transition_rate_chain_2 = 3 / 118.3       # in 1/h

    # Compute clearance
    clearance_per_allele_1 = 0.174            # in L/h
    clearance_per_allele_2 = 0.0879           # in L/h
    clearance_per_allele_3 = 0.0422           # in L/h

    if cyp2c9 == '*1/*1':
        clearance = 2 * clearance_per_allele_1
    if cyp2c9 == '*1/*2':
        clearance = clearance_per_allele_1 + clearance_per_allele_2
    if cyp2c9 == '*1/*3':
        clearance = clearance_per_allele_1 + clearance_per_allele_3
    if cyp2c9 == '*2/*2':
        clearance = 2 * clearance_per_allele_2
    if cyp2c9 == '*2/*3':
        clearance = clearance_per_allele_2 + clearance_per_allele_3
    if cyp2c9 == '*3/*3':
        clearance = 2 * clearance_per_allele_3

    change_per_year = -0.00571              # rel. change centered around 71 y
    clearance *= (1 + change_per_year * (age - 71))

    # Compute EC50 of warfarin
    ec50_per_allele_A = 0.96                # in mg/L
    ec50_per_allele_G = 2.05                # in mg/L

    if vkorc1 == 'GG':
        half_maximal_effect_conc = 2 * ec50_per_allele_G
    if vkorc1 == 'GA':
        half_maximal_effect_conc = ec50_per_allele_G + ec50_per_allele_A
    if vkorc1 == 'AA':
        half_maximal_effect_conc = 2 * ec50_per_allele_A

    parameters = [
        clearance / effective_volume_central,
        half_maximal_effect_conc,
        transition_rate_chain_1,
        transition_rate_chain_2,
        effective_volume_central
    ]

    return parameters


## Reproduce Figure 3 in Hamberg et al

In [4]:
# Define simulation conditions
times = np.linspace(0, 30, num=1000) * 24

# I: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GG VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GG'
doses = [7.6, 3.8, 1.8]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation1 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    # (Model only captures S-warfarin, so need to divide dose by 2)
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation1.append(mechanistic_model.simulate(params, times))

# II: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GA'
doses = [5.6, 2.8, 1.4]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation2 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation2.append(mechanistic_model.simulate(params, times))

# III: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'AA'
doses = [3.6, 1.8, 0.9]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation3 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation3.append(mechanistic_model.simulate(params, times))

# Figure 1: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
times /= 24
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][1],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][1],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][1],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 2: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][0],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][0],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][0],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Warfarin concentration in mg/L',
)
fig.show()

# Figure 3: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][1],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][1],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][1],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 4: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][0],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][0],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][0],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 5: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][1],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][1],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][1],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 6: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][0],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][0],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][0],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()


## Plot steady state distribution of INR for a 5mg warfarin treatment

### Generate demographic data

In [5]:
# Define population model
population_model = define_hamberg_population_model()

# Define covariates
# Frequency of alleles matches dataset in publication
n_ids = 1000
n_cov = 3
covariates = np.zeros(shape=(n_ids, n_cov))

n_cyp2p9_33 = int(np.ceil(0.006 * n_ids))
covariates[:n_cyp2p9_33, 0] += 1
n_cyp2p9_23 = int(np.ceil(0.012 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23, 0] += 1
n_cyp2p9_22 = int(np.ceil(0.014 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22, 0] += 1
n_cyp2p9_13 = int(np.ceil(0.123 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22+n_cyp2p9_13, 0] += 1
n_cyp2p9_12 = int(np.ceil(0.184 * n_ids))
covariates[
    :n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22+n_cyp2p9_13+n_cyp2p9_12, 0] += 1

typical_age = 68
covariates[:, 1] = np.random.lognormal(
    mean=np.log(typical_age), sigma=0.1, size=n_ids)

n_vkorc1_AA = int(np.ceil(0.15 * n_ids))
covariates[:n_vkorc1_AA, 2] += 1
n_vkorc1_GA = int(np.ceil(0.485 * n_ids))
covariates[:n_vkorc1_AA+n_vkorc1_GA, 2] += 1

# Sample individual-level parameters
seed = 2
data_generating_parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in population_model.get_parameter_names()
])
psi = population_model.sample(
    parameters=data_generating_parameters, covariates=covariates,
    n_samples=n_ids, seed=seed)

### Visualise demographic data

In [9]:
# Plot volume of distribution
fig = ff.create_distplot(
    [psi[:, 4]], ['Patients'], bin_size=.2, show_hist=True,
    show_curve=False)
fig.update_layout(xaxis_title='Volume of distribution in L')
fig.show()

# Plot clearance
n_1 = n_cyp2p9_33
n_2 = n_cyp2p9_33 + n_cyp2p9_23
n_3 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22
n_4 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22 + n_cyp2p9_13
n_5 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22 + n_cyp2p9_13 + n_cyp2p9_12
fig = ff.create_distplot(
    [
        psi[:n_1, 0],
        psi[n_1:n_2, 0],
        psi[n_2:n_3, 0],
        psi[n_3:n_4, 0],
        psi[n_4:n_5, 0],
        psi[n_5:, 0]
    ],
    ['*3/*3', '*2/*3', '*2/*2', '*1/*3', '*1/*2', '*1/*1'], bin_size=.001,
    show_hist=True, show_curve=False)
fig.update_layout(xaxis_title='Elimination rate in 1/h')
fig.show()

# Plot age distribution
fig = ff.create_distplot(
    [covariates[:, 1]], ['Patients'], show_hist=True,
    show_curve=False)
fig.update_layout(xaxis_title='Age in years')
fig.show()

# Plot EC50
fig = ff.create_distplot(
    [
        psi[:n_vkorc1_AA, 1],
        psi[n_vkorc1_AA:n_vkorc1_GA, 1],
        psi[n_vkorc1_AA+n_vkorc1_GA:, 1]
    ],
    ['AA', 'GA', 'GG'], bin_size=.1, show_hist=True, show_curve=False)
fig.update_layout(xaxis_title='EC50 in mg/L')
fig.show()

### Plot steady state INR distribution

In [10]:
# Define measurement model for an individual
error_models = [
    chi.LogNormalErrorModel(),  # Warfarin concentration
    chi.LogNormalErrorModel()]  # INR
predictive_model = chi.PredictiveModel(mechanistic_model, error_models)

# Select patients
patients = np.copy(psi)
patient_cov = np.copy(covariates)

# Shuffle covariates
seed = 12
indices = np.random.choice(
    np.arange(n_ids), replace=False, size=n_ids)
patients[:, 1] = patients[indices, 1]
patient_cov[:, 0] = patient_cov[indices, 0]

# Sample INR measurements
seed = 42
times = [15 * 24]
predictive_model.set_dosing_regimen(dose=5, start=0, period=24)
inr = []
for idp, patient in enumerate(patients):
    meas = predictive_model.sample(
        parameters=patient, times=times, return_df=False, seed=seed+idp)
    inr.append(meas[1, 0, 0])

# Visualise steady state disttribution
fig = ff.create_distplot(
    [inr], ['Patients'], show_hist=True, bin_size=.1, show_curve=False)
fig.update_layout(xaxis_title='INR')
fig.show()

## Check that inference can recover the model parameters

### Generate INR data

In [11]:
# Define measurement model for an individual
error_models = [
    chi.LogNormalErrorModel(),  # Warfarin concentration
    chi.LogNormalErrorModel()]  # INR
predictive_model = chi.PredictiveModel(mechanistic_model, error_models)

# Select patients randomly
seed = 1
np.random.seed(seed)
n_measured_individuals = 100
indices = np.random.choice(
    np.arange(n_ids), replace=False, size=n_measured_individuals)
selected_patients = np.copy(patients[indices])
selected_cov = np.copy(patient_cov[indices])

# Sample INR measurements
seed = 42
times = np.linspace(0, 15, num=100) * 24
predictive_model.set_dosing_regimen(dose=8, start=0, period=24)
measurements = pd.DataFrame(
    columns=['ID', 'Time', 'Observable', 'Value', 'Dose', 'Duration'])
for idp, patient in enumerate(selected_patients):
    data = predictive_model.sample(
        parameters=patient, times=times, include_regimen=True, seed=seed+idp)
    data = pd.concat([data, pd.DataFrame({
        'ID': idp + 1,
        'Time': np.nan,
        'Observable': ['CYP2C9', 'Age', 'VKORC1'],
        'Value': selected_cov[idp],
        })])
    data['ID'] = idp + 1
    measurements = pd.concat([measurements, data])

# Export data
directory = os.path.dirname(os.getcwd())
measurements.to_csv(directory + '/data/synthetic_hamberg_model_data.csv')

### Visualise INR measurements

In [12]:
# Visualise measurements
temp = measurements.copy()

# Get only 10 individuals as example
ids = temp.ID.dropna().unique()
mask = temp.ID == ids[0]
for _id in ids[np.random.randint(0, len(ids), size=10)]:
    mask = mask | (temp.ID == _id)
temp = temp[mask]
temp.ID = temp.ID / 24

fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(temp, observable='myokit.concentration_central_compartment')
fig.set_axis_labels(xlabel='Time in days', ylabel='Warfarin conc. in mg/L')
fig.show()

fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(temp, observable='myokit.inr')
fig.set_axis_labels(xlabel='Time in days', ylabel='INR')
fig.show()

## Visualise inference results

In [28]:
from model import define_hamberg_model

parameters = np.array([0.243, 4.1, 0.1, 0.02, 14.3])
times = np.linspace(0, 15, num=100) * 24
model, _ = define_hamberg_model()
model.set_dosing_regimen(8, period=24)
model.simulate(parameters, times)[1, -1]

1.1301264128953905

In [29]:
test = model.copy()
test.simulate(parameters, times)[1, -1]

1.1301264128953905

In [9]:
import pints

In [10]:
p = pints.ComposedLogPrior(
        pints.GaussianLogPrior(-3, 0.5),     # Mean log clearance
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log clearance
        pints.LogNormalLogPrior(-1.6, 0.8),  # Rel. shift clearance CYP29P *2
        pints.LogNormalLogPrior(-1.6, 0.8),  # Rel. shift clearance CYP29P *3
        pints.GaussianLogPrior(0, 0.01),     # Rel. shift clearance Age
        pints.GaussianLogPrior(1.41, 0.5),   # Mean log EC50
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log EC50
        pints.LogNormalLogPrior(-1.6, 0.8),  # Rel. shift EC50 VKORC1 A
        pints.LogNormalLogPrior(-2.3, 0.8),  # Pooled rate chain 1
        pints.LogNormalLogPrior(-3.7, 1.5),  # Pooled rate chain 2
        pints.GaussianLogPrior(2.7, 0.5),    # Mean log volume
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log volume
        pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log drug conc.
        pints.LogNormalLogPrior(0.1, 0.3)    # Sigma log INR
    )

test = p.sample()[0]
population_model.sample(test, covariates=covariates[0])

array([[0.01478298, 1.31804717, 0.06415563, 0.44374128, 8.85217633,
        0.77127681, 0.90385174]])

In [11]:
population_model.sample(test, covariates=covariates[0])

array([[ 0.01950968,  9.8793552 ,  0.06415563,  0.44374128, 17.31927678,
         0.77127681,  0.90385174]])

In [50]:
import chi
import numpy as np
import pandas as pd
import pints
from model import define_hamberg_model, define_hamberg_population_model

In [58]:
# Import data
directory = os.path.dirname(os.getcwd())
measurements_df = pd.read_csv(
    directory + '/data/synthetic_hamberg_model_data.csv')

# Define hierarchical log-posterior
mechanistic_model, params = define_hamberg_model()
error_models = [chi.GaussianErrorModel(), chi.GaussianErrorModel()]
population_model = define_hamberg_population_model(centered=False)
log_prior = pints.ComposedLogPrior(
    pints.GaussianLogPrior(-3.7, 0.5),   # Mean log elim. rate
    pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log elim. rate
    pints.UniformLogPrior(0, 1),         # Rel. shift elim. rate *2/*2
    pints.UniformLogPrior(0, 1),         # Rel. shift elim. rate *3/*3
    pints.LogNormalLogPrior(-3, 1.4),    # Rel. shift elim. rate Age
    pints.GaussianLogPrior(1.41, 0.5),   # Mean log EC50
    pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log EC50
    pints.UniformLogPrior(0, 1),         # Rel. shift EC50 A/A
    pints.LogNormalLogPrior(-2.3, 0.7),  # Pooled rate chain 1
    pints.LogNormalLogPrior(-3.7, 1.5),  # Pooled rate chain 2
    pints.GaussianLogPrior(2.7, 0.5),    # Mean log volume
    pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log volume
    pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log drug conc.
    pints.LogNormalLogPrior(0.1, 0.3)    # Sigma log INR
)
log_prior = pints.UniformLogPrior([0]*population_model.n_parameters(), [100]*population_model.n_parameters())
problem = chi.ProblemModellingController(mechanistic_model, error_models)
problem.set_population_model(population_model)
problem.set_data(measurements_df)
problem.set_log_prior(log_prior)

posterior =  problem.get_log_posterior()

In [116]:
log_prior = pints.ComposedLogPrior(
    pints.GaussianLogPrior(-3.7, 0.5),   # Mean log elim. rate
    pints.LogNormalLogPrior(-1, 0.3),    # Sigma log elim. rate
    pints.UniformLogPrior(0, 1),         # Rel. shift elim. rate *2/*2
    pints.UniformLogPrior(0, 1),         # Rel. shift elim. rate *3/*3
    pints.LogNormalLogPrior(-3, 1.4),    # Rel. shift elim. rate Age
    pints.GaussianLogPrior(1.41, 0.3),   # Mean log EC50
    pints.LogNormalLogPrior(0.1, 0.2),   # Sigma log EC50
    pints.UniformLogPrior(0, 1),         # Rel. shift EC50 A/A
    pints.LogNormalLogPrior(-2.3, 0.7),  # Pooled rate chain 1
    pints.LogNormalLogPrior(-3.7, 1.5),  # Pooled rate chain 2
    pints.GaussianLogPrior(2.7, 0.1),    # Mean log volume
    pints.LogNormalLogPrior(-1, 0.1),    # Sigma log volume
    pints.LogNormalLogPrior(0.1, 0.3),   # Sigma log drug conc.
    pints.LogNormalLogPrior(0.1, 0.3)    # Sigma log INR
)
posterior._log_prior = log_prior

In [105]:
m = 0.05
s = 0.1
np.exp(s**2 + 2 * m)*(np.exp(s**2)-1)

0.011218781120504258

In [115]:
log_prior.sample()

array([[-3.09240311,  1.21346885,  0.04257891,  0.4956266 ,  0.04544251,
         2.14166638,  0.89701082,  0.86818369,  0.16264095,  0.28897093,
         2.77853204,  0.36682905,  1.00003858,  1.29043678]])

In [108]:
samples[0, 300:]

array([-3.8707311 ,  0.75139511,  0.2847048 ,  0.32013056,  0.10965009,
        0.54939157,  0.97802067,  0.74888604,  0.06628501,  0.0635197 ,
        2.33766734,  1.05223134,  1.37164079,  0.74381006])

In [118]:
samples = posterior.sample_initial_parameters(n_samples=100)
for s in samples:
    score = posterior(s)
    if np.isinf(score):
        print('Mean and var elimination rate: ', [np.exp(s[300]), np.exp(s[301]**2 + 2 * s[300])*(np.exp(s[301]**2)-1)])
        print('Shifts elimination rate: ', s[302:305])
        print('Mean and var EC50: ', [np.exp(s[305]), np.exp(s[306]**2 + 2 * s[305])*(np.exp(s[306]**2)-1)])
        print('Shift EC50: ', s[307])
        print('Transition rate: ', s[308:310])
        print('Mean and var Volume: ', [np.exp(s[310]), np.exp(s[311]**2 + 2 * s[310])*(np.exp(s[311]**2)-1)])

/Users/david/Workspace/sabs-r3/mipd-warfarin/results/2_semi_mechanistic_model/model.py:682: RuntimeWarning: divide by zero encountered in log
  np.log(1 - np.tanh(age[:, np.newaxis] * age_change))


In [17]:
epsilon = 0.0000001
n_parameters = posterior.n_parameters()
parameters = np.array([14.3, 0.348, 4.1, 0.1, 0.02, 0.1, 0.1])
ref_sens = []
posterior(parameters)
for index in range(n_parameters):
    # Construct parameter grid
    low = parameters.copy()
    low[index] -= epsilon
    high = parameters.copy()
    high[index] += epsilon

    # Compute reference using numpy.gradient
    sens = np.gradient(
        [
            posterior(low),
            posterior(parameters),
            posterior(high)],
        (epsilon))
    ref_sens.append(sens[1])

# Compute sensitivities with hierarchical model
for l in posterior._log_likelihood._log_likelihoods:
    l._mechanistic_model._mechanistic_model._simulator.set_tolerance(
        abs_tol=1e-11, rel_tol=0.00000000001)
_, sens = posterior.evaluateS1(parameters)

/Users/david/Workspace/sabs-r3/mipd-warfarin/venv/lib/python3.9/site-packages/chi/_log_pdfs.py:963: RuntimeWarning:

An error occured while solving the mechanistic model: 
can't convert complex to float.
 A score of -infinity is returned.

/Users/david/Workspace/sabs-r3/mipd-warfarin/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:1235: RuntimeWarning:

invalid value encountered in subtract

/Users/david/Workspace/sabs-r3/mipd-warfarin/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:1256: RuntimeWarning:

invalid value encountered in double_scalars

/Users/david/Workspace/sabs-r3/mipd-warfarin/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:1263: RuntimeWarning:

invalid value encountered in double_scalars

/Users/david/Workspace/sabs-r3/mipd-warfarin/venv/lib/python3.9/site-packages/chi/_log_pdfs.py:1137: RuntimeWarning:

An error occured while solving the mechanistic model: 
can't convert complex to float.
 A score of -infinity is returned.



In [22]:
sens

array([ 1.21990782e-01,  1.10340366e+02,  9.83133318e+00, -1.12649406e+01,
       -2.49239354e+02,  9.39303874e+07,  8.91763403e+04])

In [23]:
ref_sens

[0.28870999813079834,
 110.24530977010727,
 9.960494935512543,
 -3.548339009284973,
 -254.43732738494873,
 93930388.87064904,
 89176.3437539339]

In [45]:
posterior(parameters)

-6022.74357490803

In [29]:
epsilon = 0.00001
n_parameters = model.n_parameters()
parameters = np.full(shape=n_parameters, fill_value=0.1)
observations = np.random.lognormal(0.1, 0.1, size=3)
covariates = np.arange(3)[:, np.newaxis]
ref_sens = []
for index in range(3):
    # Construct parameter grid
    low = observations.copy()
    low[index] -= epsilon
    high = observations.copy()
    high[index] += epsilon

    # Compute reference using numpy.gradient
    sens = np.gradient(
        [
            model.compute_log_likelihood(parameters, low, covariates),
            model.compute_log_likelihood(parameters, observations, covariates),
            model.compute_log_likelihood(parameters, high, covariates)],
        (epsilon))
    ref_sens.append(sens[1])

# Compute sensitivities with hierarchical model
_, dobs, sens = model.compute_sensitivities(parameters, observations, covariates)

In [30]:
ref_sens

[-3.862263783371844, -18.746957540616283, -5.13508099538873]

In [31]:
dobs

array([[ -3.86226379],
       [-18.74695754],
       [ -5.135081  ]])

In [1]:
from model import define_hamberg_model
model, _ = define_hamberg_model()

In [3]:
model.parameters()

['myokit.absorption_rate',
 'myokit.elimination_rate',
 'myokit.half_maximal_effect_concentration',
 'myokit.transition_rate_chain_1',
 'myokit.transition_rate_chain_2',
 'myokit.volume']

In [15]:
from model import define_hamberg_model
import numpy as np
import chi

parameters = np.array([0.243, 4.1, 0.1, 0.02, 14.3, 1, 1])
times = np.linspace(0, 15, num=100) * 24
model, _ = define_hamberg_model()
model.set_dosing_regimen(8, period=24)
model.set_outputs(['myokit.inr'])
observations = model.simulate(parameters[:-2], times)
error_model = chi.GaussianErrorModel() #[chi.GaussianErrorModel(), chi.GaussianErrorModel()]
log_likelihood = chi.LogLikelihood(model, error_model, observations, times)#[times, times])


In [16]:
epsilon = 0.000001
parameters = np.array([0.2, 4.3, 0.12, 0.042, 14, 1, 1])
n_parameters = log_likelihood.n_parameters()
ref_sens = []
for index in range(n_parameters):
    # Construct parameter grid
    low = parameters.copy()
    low[index] -= epsilon
    high = parameters.copy()
    high[index] += epsilon

    # Compute reference using numpy.gradient
    sens = np.gradient(
        [
            log_likelihood(low),
            log_likelihood(parameters),
            log_likelihood(high)],
        (epsilon))
    ref_sens.append(sens[1])

# Compute sensitivities with hierarchical model
_, sens = log_likelihood.evaluateS1(parameters)

In [12]:
log_likelihood.get_parameter_names()

['myokit.elimination_rate',
 'myokit.half_maximal_effect_concentration',
 'myokit.transition_rate_chain_1',
 'myokit.transition_rate_chain_2',
 'myokit.volume',
 'Sigma']

In [17]:
np.array(ref_sens)

array([  2.64740187,   0.15057154,  -0.18679297,  -1.60155061,
        -0.46476152, -99.85547337])

In [18]:
sens

array([ 2.61036196e+00,  1.35241509e-01, -1.38796785e-01, -1.62691447e+00,
        4.12800236e-02, -9.98554734e+01])

In [8]:
model.enable_sensitivities(True)
np.sum(model.simulate(parameters[:-2], times)[1], axis=0)

array([[-29.38857775,   0.        ,   0.        ,   0.        ,
         -0.41380578]])

In [11]:
print(model._mechanistic_model._model.code())

[[model]]
name: hamberg_warfarin_inr_model_with_sensitivities
# Initial values
myokit.delay_compartment_1_chain_1                                     = 1
myokit.delay_compartment_1_chain_2                                     = 1
myokit.delay_compartment_2_chain_1                                     = 1
myokit.delay_compartment_2_chain_2                                     = 1
myokit.relative_change_cf1                                             = 1
myokit.relative_change_cf2                                             = 1
myokit.amount_dose_compartment                                         = 1
myokit.amount_central_compartment                                      = 1
myokit.damount_central_compartment_delimination_rate                   = 1
myokit.drelative_change_cf1_dvolume                                    = 1
myokit.drelative_change_cf1_delimination_rate                          = 1
myokit.drelative_change_cf1_dhalf_maximal_effect_concentration         = 1
myokit.drelative_chan

In [28]:
np.tanh(10)

0.9999999958776927

In [48]:
np.exp(4.2)

66.68633104092515

In [46]:
s = 0.2
np.exp(s**2 + 2 * 2.7)*(np.exp(s**2)-1)

9.404523913613165